# DataStorm 2.0

## Import Libraries

In [2]:
import pandas as pd

## Import Data Sets

In [3]:
train_data = pd.read_csv('../Data/Hotel-A-train.csv')

validation_data = pd.read_csv('../Data/Hotel-A-validation.csv')

test_data = pd.read_csv('../Data/Hotel-A-test.csv')

train_data.head(10)

,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Expected_checkin,Expected_checkout,...,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion,Discount_Rate,Room_Rate
0,39428300,F,40,Latino,Grad,<25K,North,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,No Deposit,Online,Yes,Check-In,Yes,10,218
1,77491756,F,49,Latino,Mid-School,50K -- 100K,East,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,Refundable,Online,Yes,Check-In,No,0,185
2,73747291,F,42,caucasian,Grad,<25K,East,City Hotel,7/2/2015,7/6/2015,...,BB,No,No,No Deposit,Online,Yes,Check-In,No,0,119
3,67301739,M,25,African American,College,>100K,South,Airport Hotels,7/2/2015,7/3/2015,...,BB,No,No,Refundable,Agent,Yes,Check-In,Yes,5,144
4,77222321,F,62,Latino,High-School,25K --50K,East,Resort,7/3/2015,7/4/2015,...,BB,No,No,No Deposit,Direct,No,Check-In,Yes,10,242
5,55152245,M,34,African American,College,>100K,North,Airport Hotels,7/3/2015,7/4/2015,...,BB,Yes,No,No Deposit,Online,No,Check-In,Yes,10,143
6,87139343,F,53,African American,High-School,<25K,East,Airport Hotels,7/3/2015,7/4/2015,...,BB,Yes,Yes,Refundable,Online,Yes,Check-In,Yes,25,212
7,7647912,M,40,Latino,Mid-School,>100K,North,City Hotel,7/3/2015,7/4/2015,...,FB,No,No,No Deposit,Online,Yes,Check-In,No,0,170
8,94296865,M,59,African American,Mid-School,>100K,West,Resort,5/6/2015,5/7/2015,...,BB,No,No,Non-Refundable,Direct,Yes,Check-In,Yes,10,245
9,93087487,M,21,Latino,Grad,50K -- 100K,West,City Hotel,4/22/2015,4/25/2015,...,HB,Yes,No,Refundable,Online,No,Check-In,Yes,20,212


In [4]:
train_headers = train_data.columns
print(len(train_headers))
train_headers

24


Index(['Reservation-id', 'Gender', 'Age', 'Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type', 'Expected_checkin',
       'Expected_checkout', 'Booking_date', 'Adults', 'Children', 'Babies',
       'Meal_Type', 'Visted_Previously', 'Previous_Cancellations',
       'Deposit_type', 'Booking_channel', 'Required_Car_Parking',
       'Reservation_Status', 'Use_Promotion', 'Discount_Rate', 'Room_Rate'],
      dtype='object')

## Data Formatting

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

# columns
cat_data = ['Gender','Ethnicity','Educational_Level','Income','Country_region','Hotel_Type','Meal_Type','Visted_Previously','Previous_Cancellations','Deposit_type','Booking_channel','Required_Car_Parking','Reservation_Status','Use_Promotion']
num_data = ['Age', 'Adults', 'Children', 'Babies', 'Discount_Rate', 'Room_Rate']
y_data = ['Reservation_Status_Canceled','Reservation_Status_Check-In','Reservation_Status_No-Show']

drop_data = ['Reservation-id','Expected_checkin', 'Expected_checkout', 'Booking_date']

for element in drop_data:
    if element in cat_data:
        cat_data.remove(element)
    if element in num_data:
        num_data.remove(element)

print("Total Fields =", len(cat_data + drop_data + num_data))

def format_data(data_set, is_test=False):
    if(is_test and ('Reservation_Status' in cat_data)):
        cat_data.remove('Reservation_Status')

    f_data_set = pd.get_dummies(data_set, prefix=cat_data, columns=cat_data)
    f_data_set = f_data_set.drop(drop_data, axis = 1)
    if(is_test):
        x = f_data_set
        return x
    else:
        x = f_data_set.drop(y_data, axis = 1)
        y = f_data_set[y_data]
        return x, y

x_train, y_train = format_data(train_data)
x_val, y_val = format_data(validation_data)
x_test = format_data(test_data, is_test=True)

# x_val1, x_val2, y_val1, y_val2 = train_test_split(x_val, y_val, test_size=0.8, random_state=10)

y_val.head(10)

Total Fields = 24


,Reservation_Status_Canceled,Reservation_Status_Check-In,Reservation_Status_No-Show
0,0,0,1
1,1,0,0
2,1,0,0
3,0,1,0
4,0,1,0
5,1,0,0
6,0,0,1
7,1,0,0
8,0,1,0
9,0,1,0


## Normalize Data

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
x_val = pd.DataFrame(scaler.transform(x_val), columns=x_val.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)

x_train.head(10)

,Age,Adults,Children,Babies,Discount_Rate,Room_Rate,Gender_F,Gender_M,Ethnicity_African American,Ethnicity_Asian American,...,Deposit_type_No Deposit,Deposit_type_Non-Refundable,Deposit_type_Refundable,Booking_channel_Agent,Booking_channel_Direct,Booking_channel_Online,Required_Car_Parking_No,Required_Car_Parking_Yes,Use_Promotion_No,Use_Promotion_Yes
0,0.423077,0.25,0.5,0.0,0.250,0.786667,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1,0.596154,0.50,1.0,0.0,0.000,0.566667,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.461538,0.50,1.0,0.0,0.000,0.126667,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
3,0.134615,0.75,1.0,0.0,0.125,0.293333,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.846154,0.00,0.0,0.0,0.250,0.946667,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
5,0.307692,1.00,0.5,0.5,0.250,0.286667,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
6,0.673077,0.25,0.0,0.0,0.625,0.746667,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
7,0.423077,0.25,0.0,0.0,0.000,0.466667,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
8,0.788462,0.50,0.5,0.0,0.250,0.966667,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
9,0.057692,0.25,1.0,0.0,0.500,0.746667,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [8]:
corr = x_train.join(y_train).corr()

# corr.style.background_gradient(cmap='coolwarm')

## Model Training

In [9]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=15, random_state=0)
classifier.fit(x_train, y_train)

classifier.score(x_val, y_val)

0.5394688977810113

In [10]:
from sklearn.metrics import f1_score

y_pred = classifier.predict(x_val)

f1_score(y_val, y_pred, average='macro')

0.24690423195426978

## Inference

In [11]:
y_test = classifier.predict(x_test)

y_test

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=uint8)

In [29]:

df = pd.DataFrame(y_test, columns=[2,1,3]).idxmax(axis=1)
df.index = test_data['Reservation-id']
df.to_csv('results/Submission.csv', header=['Reservation_status'])
